# CIFAR-100: Full Lab — Account setup, Train → W&B → Deploy on Hugging Face Spaces
**Colab-ready hands-on lab notebook**

This notebook combines:
- Step-by-step instructions to create a **Hugging Face** account and **access token**,
- Secure token input in Colab using `getpass` (token is not printed),
- **W&B** authentication,
- Training a `resnet18` on **CIFAR-100**, logging to W&B and saving model as an artifact,
- A Gradio app (`app.py`) that downloads the model artifact from W&B at startup,
- A script to create & push a **Hugging Face Space** (Gradio) from Colab using `huggingface_hub`.

**Security note:** Never commit your tokens/API keys to public repos. Use Colab `getpass` to keep tokens secret and use HF Space Secrets for W&B keys.


In [1]:
#@title 🔍 Environment Validation (run before starting lab)
import torch, os

print("---- Environment Validation ----")

# Check GPU availability
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

# Check required env vars
for var in ["HF_TOKEN", "HF_USER", "SPACE_NAME"]:
    val = os.environ.get(var)
    print(f"{var}:", " SET" if val else "NOT SET")

# Check if W&B API key configured (login handled separately)
wandb_key = os.environ.get("WANDB_API_KEY")
print("WANDB_API_KEY:", " SET" if wandb_key else "⚠️ Not set (you'll log in interactively)")

# Check if dependencies installed
try:
    import wandb, gradio, huggingface_hub
    print("Dependencies:  wandb, gradio, huggingface_hub imported successfully")
except Exception as e:
    print("Dependencies: missing - please run install cell first")
    print(e)

print("---------------------------------")


---- Environment Validation ----
GPU available: True
GPU name: Tesla T4
HF_TOKEN: NOT SET
HF_USER: NOT SET
SPACE_NAME: NOT SET
WANDB_API_KEY: ⚠️ Not set (you'll log in interactively)
Dependencies:  wandb, gradio, huggingface_hub imported successfully
---------------------------------


In [2]:
#@title 1) Install dependencies (run once)
# Installs pytorch (with CUDA if available), W&B, Gradio and HF tooling.
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q wandb gradio huggingface_hub git-lfs


## 2) Hugging Face account & token (student instructions)

**If you don't have a Hugging Face account:**
1. Go to https://huggingface.co and click **Sign up**. Verify email.
2. Choose a username (used as `HF_USER`).

**Create an Access Token:**
1. Go to https://huggingface.co/settings/tokens
2. Click **New token**. Name it (e.g., `colab-space-token`) and select scope `repo` (read & write).
3. Click **Create** and copy the token now — you won't see it again.

**Important:** Keep the token secret. Use the next cell to set it securely in Colab (the token will not be printed).

In [22]:
#@title 3) Securely set your Hugging Face token, username, and desired Space name
from getpass import getpass
import os

print("Paste your Hugging Face token when prompted. It will be hidden.")
hf_token = getpass("Hugging Face token: ")
os.environ['HF_TOKEN'] = hf_token

# Edit these values (do NOT put the token here)
hf_user = input("Enter your Hugging Face username (e.g. 'alice'): ").strip()
space_name = input("Enter desired Space name (e.g. 'cifar100-demo-space'): ").strip()

os.environ['HF_USER'] = hf_user
os.environ['SPACE_NAME'] = space_name

print("HF_TOKEN stored in runtime (hidden). HF_USER and SPACE_NAME saved in environment variables.")

Paste your Hugging Face token when prompted. It will be hidden.
Hugging Face token: ··········
Enter your Hugging Face username (e.g. 'alice'): vaibhav9029
Enter desired Space name (e.g. 'cifar100-demo-space'): cifar100-demo-space
HF_TOKEN stored in runtime (hidden). HF_USER and SPACE_NAME saved in environment variables.


In [9]:
#@title 4) Authenticate Weights & Biases (W&B)
import wandb
print("Follow the prompt to authenticate W&B (this opens an input box).")
wandb.login()


Follow the prompt to authenticate W&B (this opens an input box).


True

## 5) Create `train.py` — training + W&B artifact logging

This script trains ResNet18 on CIFAR-100, logs metrics to W&B, and saves the best model as a W&B artifact.


In [40]:
%%bash
cat > train.py <<'PY'
import argparse
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from torch.utils.data import DataLoader
from torchvision.models import resnet18
print("god")
def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("--project", type=str, default="cifar100-hf-demo")
    p.add_argument("--entity", type=str, default=None)
    p.add_argument("--epochs", type=int, default=5)
    p.add_argument("--batch-size", type=int, default=128)
    p.add_argument("--lr", type=float, default=0.01)
    return p.parse_args()

def get_dataloaders(batch_size):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409),
                             (0.2673, 0.2564, 0.2762)),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4865, 0.4409),
                             (0.2673, 0.2564, 0.2762)),
    ])
    trainset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform_train)
    testset  = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_test)
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader  = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    return trainloader, testloader

def train_one_epoch(model, device, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return running_loss / total, 100. * correct / total

def evaluate(model, device, loader, criterion):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            l = criterion(outputs, targets)
            loss += l.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return loss/total, 100.*correct/total

def main():
    args = parse_args()
    wandb.init(project=args.project, entity=args.entity, config=vars(args))
    cfg = wandb.config

    device = "cuda" if torch.cuda.is_available() else "cpu"
    trainloader, testloader = get_dataloaders(cfg.batch_size)

    model = resnet18(num_classes=100)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=cfg.lr, momentum=0.9, weight_decay=5e-4)

    best_acc = 0.0
    for epoch in range(cfg.epochs):
        train_loss, train_acc = train_one_epoch(model, device, trainloader, optimizer, criterion)
        test_loss, test_acc = evaluate(model, device, testloader, criterion)
        wandb.log({"epoch": epoch+1, "train_loss": train_loss, "train_acc": train_acc,
                   "test_loss": test_loss, "test_acc": test_acc})
        print(f"Epoch {epoch+1}: train_acc={train_acc:.2f} test_acc={test_acc:.2f}")
        if test_acc > best_acc:
            best_acc = test_acc
            os.makedirs("outputs", exist_ok=True)
            torch.save(model.state_dict(), "outputs/model.pt")
            # log artifact
            artifact = wandb.Artifact("resnet18-cifar100", type="model", metadata={"test_acc": best_acc})
            artifact.add_file("outputs/model.pt")
            wandb.log_artifact(artifact)
    print("Best test acc:", best_acc)

if __name__ == "__main__":
    main()
PY


In [15]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [18]:
#@title 6) Run training (edit the --entity to your W&B username/team)
# Keep epochs small for demo (e.g., 3-5). Increase for better accuracy.
!python train.py --project cifar100-hf-demo --entity ir2023 --epochs 3 --batch-size 128


wandb: Currently logged in as: 142502033 (ir2023) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.22.2
wandb: Run data is saved locally in /content/wandb/run-20251021_043557-pe4565kz
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run lunar-blaze-3
wandb: ⭐️ View project at https://wandb.ai/ir2023/cifar100-hf-demo
wandb: 🚀 View run at https://wandb.ai/ir2023/cifar100-hf-demo/runs/pe4565kz
100% 169M/169M [00:04<00:00, 41.5MB/s]
Epoch 1: train_acc=10.56 test_acc=17.55
Epoch 2: train_acc=19.53 test_acc=24.56
Epoch 3: train_acc=24.89 test_acc=28.80
Best test acc: 28.8
wandb: 
wandb: 🚀 View run lunar-blaze-3 at: https://wandb.ai/ir2023/cifar100-hf-demo/runs/pe4565kz
wandb: Find logs at: wandb/run-20251021_043557-pe4565kz/logs


## 7) Download model artifact from W&B (if you didn't run training here)
This downloads the latest logged artifact into `outputs/` so the Gradio app can load it.

In [24]:
%%bash
python - <<'PY'
import wandb, os, sys
ENTITY = os.environ.get("WANDB_ENTITY") or "ir2023"   # <-- edit if not set
PROJECT = "cifar100-hf-demo"
ARTIFACT = "resnet18-cifar100:latest"
api = wandb.Api()
try:
    artifact = api.artifact(f"{ENTITY}/{PROJECT}/{ARTIFACT}")
    artifact.download(root="outputs")
    print("Downloaded artifact to outputs/")
except Exception as e:
    print("Failed to download artifact:", e)
    sys.exit(1)
PY


Downloaded artifact to outputs/


wandb:   1 of 1 files downloaded.  


## 8) Create Gradio app (`app.py`) that downloads the model from W&B at startup

This approach avoids committing large model files into the Space repository. The Space must have `WANDB_API_KEY` set as a secret in its settings.


In [35]:
import os, time, io
from PIL import Image
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
import gradio as gr

MODEL_PATH = "outputs/model.pt"

# If model not present, try download via W&B (requires WANDB_API_KEY secret in Space or env)
if not os.path.exists(MODEL_PATH):

    try:
        import wandb
        wandb_api_key = os.environ.get("WANDB_API_KEY")
        if wandb_api_key:
            wandb.login(key=wandb_api_key)
            api = wandb.Api()
            artifact = api.artifact(os.environ.get("WANDB_ARTIFACT", "ir2023/cifar100-hf-demo/resnet18-cifar100:v12"))
            artifact.download(root="outputs")
            print("Downloaded model via W&B artifact.")
        else:
            print("WANDB_API_KEY not set; cannot download artifact.")
    except Exception as e:
        print("Error downloading artifact via W&B:", e)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = resnet18(num_classes=100)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409),(0.2673,0.2564,0.2762))
])

def predict_image(img):
    start = time.time()
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        probs = torch.nn.functional.softmax(out, dim=1)
        conf, idx = probs.max(1)
        class_idx = int(idx.item())
        conf_val = float(conf.item())
    latency = (time.time() - start) * 1000.0
    return {"class_idx": class_idx, "confidence": round(conf_val,4), "latency_ms": round(latency,2)}

iface = gr.Interface(fn=predict_image, inputs=gr.Image(type="pil"), outputs="json", title="CIFAR-100 demo")
if __name__ == "__main__":
    iface.launch(server_name="0.0.0.0", server_port=7860)



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cad6d603423269ee93.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [36]:
%%bash
cat > requirements.txt <<'REQ'
torch
torchvision
gradio
Pillow
wandb
huggingface_hub
git-lfs
REQ


## 9) Create & push a Hugging Face Space from Colab (uses HF_TOKEN set earlier via getpass)

This will:
- create the Space repo (if it doesn't exist),
- push `app.py` and `requirements.txt` to the Space,
- not include the heavy model file (the app downloads it from W&B at runtime).


In [37]:
%%bash
set -e
# prepare local repo
rm -rf hf_space || true
mkdir hf_space
cp app.py requirements.txt hf_space/
cd hf_space

git init
git config user.email "142502033@smail.iitpkd.ac.in"
git config user.name "Vaibhav9029"
git lfs install

python - <<'PY'
from huggingface_hub import HfApi, Repository
import os, sys
token = os.environ.get("HF_TOKEN")
user = os.environ.get("HF_USER")
space = os.environ.get("SPACE_NAME")
if not token or not user or not space:
    print("HF_TOKEN, HF_USER or SPACE_NAME not set. Aborting.")
    sys.exit(1)
api = HfApi(token=token)

repo_id = f"{user}/{space}"
repo_url = api.create_repo(repo_id=repo_id, repo_type="space",
            space_sdk="gradio",
            exist_ok=True)
print("Repo URL:", repo_url)

api.upload_folder(
    folder_path=".",
    repo_id=repo_id,
    repo_type="space",
    commit_message="Initial commit: CIFAR-100 Gradio app (no model)"
)

print("Pushed to:", repo_url)
PY


Initialized empty Git repository in /content/hf_space/.git/
Updated git hooks.
Git LFS initialized.
Repo URL: https://huggingface.co/spaces/vaibhav9029/cifar100-demo-space
Pushed to: https://huggingface.co/spaces/vaibhav9029/cifar100-demo-space


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>


## 10) Adding W&B API key as a secret in the Hugging Face Space

After pushing the Space, do:
1. Visit `https://huggingface.co/spaces/<HF_USER>/<SPACE_NAME>`.
2. Click **Settings → Secrets**.
3. Add a secret named `WANDB_API_KEY` and paste your W&B API key (keep secret).
4. Optionally set `WANDB_ARTIFACT` to the artifact path, e.g. `your_wandb_entity/cifar100-hf-demo/resnet18-cifar100:latest`.
5. Rebuild the Space (there's a "Rebuild" button) so the app can download the artifact at startup.


## 11) Test the deployed Space
Open the Space URL `https://huggingface.co/spaces/<HF_USER>/<SPACE_NAME>`.
- Check the Space logs (Settings → Logs) if the model download or runtime fails.
- Common issues: missing secret, wrong artifact name, package install errors (pin versions in requirements.txt).


## 12) Instructor tips and cleanup
- Keep epochs small during demos.
- Instructor can host one shared Space and W&B project to avoid per-student pushes.
- After course, remove the Space or set it to private if not needed.
- Revoke tokens if accidentally exposed: Hugging Face (Settings → Access Tokens), W&B (Settings → API Keys).
